In [ ]:
!pip install --upgrade boto3
!pip install --upgrade sagemaker

In [ ]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

print(role)
print(bucket)

if "cn-" in region_name:
    with open('./code/requirements.txt', 'r') as original: data = original.read()
    with open('./code/requirements.txt', 'w') as modified: modified.write("-i https://pypi.tuna.tsinghua.edu.cn/simple\n" + data)

!touch dummy
!tar czvf model.tar.gz dummy
assets_dir = 's3://{0}/{1}/assets/'.format(bucket, 'llm_chinese')
model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(bucket, 'llm_chinese')
!aws s3 cp model.tar.gz $assets_dir
!rm -f dummy model.tar.gz

model_name = None
entry_point = 'inference.py'
framework_version = '1.13.1'
py_version = 'py39'
model_environment = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'600', 
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
}

from sagemaker.pytorch.model import PyTorchModel

model = PyTorchModel(
    name = model_name,
    model_data = model_data,
    entry_point = entry_point,
    source_dir = './code',
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

endpoint_name = 'pytorch-inference-llm-v1'
# instance_type = 'ml.p3.2xlarge'
instance_type='ml.g4dn.2xlarge' 

instance_count = 1

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count,
    serializer = JSONSerializer(),
    deserializer = JSONDeserializer()
)

### 测试


In [ ]:
#休眠2分钟,确保模型可以完全加载
import time
time.sleep(120)

In [ ]:
inputs= {
    "ask": "你好!"

}

response = predictor.predict(inputs)
print(response["answer"])

inputs= {
    "ask": "晚上睡不着应该怎么办"

}

response = predictor.predict(inputs)
print(response["answer"])

### 如果上述测试给出答案并且没有报错，则该模型部署已经成功部署，您可以关闭该脚本，去完成其他步骤。
### 下面是可选测试项，仅供参考（并非必选）

In [ ]:
inputs= {
    "ask": "怎么修改huggingface transformers的model cache位置"

}

response = predictor.predict(inputs)
print(response["answer"])

inputs= {
    "ask": "用python3写出快速排序代码"

}

response = predictor.predict(inputs)
print(response["answer"])

In [ ]:
#我们来查看一下刚部署的这个模型对应的SageMaker inference endpoint
sagemaker_endpoint_name = predictor.endpoint_name
sagemaker_endpoint_name

利用已经在SageMaker real time inference endpoint部署的LLM模型来模拟单轮对话和多轮对话。

下面的代码建议在SageMaker Notebook上来运行。

In [ ]:
import json
import boto3

client = boto3.client('runtime.sagemaker')

def query_endpoint_with_json_payload(encoded_json):
    response = client.invoke_endpoint(EndpointName=sagemaker_endpoint_name, ContentType='application/json', Body=encoded_json)
    return response

def parse_response_texts(query_response):
    model_predictions = json.loads(query_response['Body'].read())
    generated_text = model_predictions["answer"]
    return generated_text


先简单测试一下针对单个问题的回答

In [ ]:
payload = {"ask": "信息抽取：\n2022年世界杯的冠军是阿根廷队伍，梅西是MVP\n问题：国家名，人名\n答案："}
query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
generated_texts = parse_response_texts(query_response)
print(generated_texts)

单轮对话：每个问题/query都是独立的，问题之间没有关联性。

In [ ]:
#1.首先需要一个简单的开场拍。
print("用户：你好！\nAI：您好!我是AI。我可以回答您的问题、写文章、写作业、翻译，对于一些法律等领域的问题我也可以给你提供信息。")

#2.在同一个session中持续对话，但是每次对话之间没有什么关联。
while True:
    command = input("用户：")
    if command == 'quit':
        break;
    
    #print(command)
    payload = {"ask": "\n用户："+ command + "\n"}
    #print(payload["ask"])
    query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
    generated_texts = "AI：" + parse_response_texts(query_response)
    print(generated_texts)

多轮对话模拟：我们这里来测试一下多轮对话能力。

In [ ]:
#1.首先需要开场拍来引导AI，其实就是给它一个上下文来启动所谓的对话session。
payload = {"ask": "用户：你好！\nAI：您好!我是AI。我可以回答您的问题、写文章、写作业、翻译，对于一些法律等领域的问题我也可以给你提供信息。"}
print(payload["ask"])
generated_texts = ""

#在这里简单模拟多轮对话时，发送给SageMaker endpoint的payload会越来越大，这里对payload大致做一个限制。
session_len = 10 * 1024 * 1024 

#2.在同一个session中持续对话，为了有多轮对话的效果，把每一轮的信息(问题-回答对)都带上来告诉LLM之前的上下文。
while True:
    command = input("用户：")
    if command == 'quit':
        break;
    
    #print(command)
    whole_context = payload["ask"] + generated_texts + "\n用户："+ command + "\n"
    payload = {"ask": whole_context}
    if len(whole_context) > session_len:
        print("上下文信息太多了，当前对话session要退出了！")
        break;
    #print(payload["ask"])
    query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
    generated_texts = "AI：" + parse_response_texts(query_response)
    print(generated_texts)

### 删除SageMaker  Endpoint
删除推理服务

In [ ]:
predictor.delete_endpoint()